In [1]:
import ezdxf
from dxfgrouping import filterlayerelements, makemergevertset, linearizeelement
from dxfgrouping import getblockcomponent, dxfoutputblocks

def convertall(inputfilename, outputfilename, cutlayers, penlayers, breflY, dmax):
    d = ezdxf.readfile(inputfilename)
    print("layerspresent:", set(e.dxf.layer for e in d.entities))
    d = ezdxf.readfile(inputfilename)
    lwpolylines = [ e  for e in d.entities  if e.dxftype() == "LWPOLYLINE" ]
    if lwpolylines:
        #print("exploding %d lwpolylines" % len(lwpolylines))
        for e in lwpolylines:
            e.explode()

    cutelements = filterlayerelements(d, cutlayers)
    penelements = filterlayerelements(d, penlayers)
        
    lcutelements, lpenelements = cutelements, penelements
    elementgroups = [ ]

    while lcutelements:
        res = getblockcomponent(lcutelements, lpenelements, dmax)
        (outercutelements, outercutelementsdir) = res[0]
        (internalcutelements, internalpenelements) = res[1]
        (remainingcutelements, remainingpenelements) = res[2]
        elementgroups.append((outercutelements, outercutelementsdir, internalcutelements, internalpenelements))
        lcutelements, lpenelements = remainingcutelements, remainingpenelements
        #print("Component: ", len(outercutelements), len(internalcutelements), len(internalpenelements))
        
    dxfoutputblocks(outputfilename, elementgroups, breflY)
    return elementgroups


In [2]:
import os
def createorclear(aamaoutdir, lsubdir):
    subdir = os.path.join(aamaoutdir, lsubdir)
    if os.path.exists(subdir):
        for f in os.listdir(subdir):
            os.remove(os.path.join(subdir, f))
    else:
        os.mkdir(subdir)
    return subdir
        


In [4]:
#-2R15-BS-01- Bottom surface. 
#  BS-CUT layer to be cut, BS-PLOT to be plotted, all pieces cut from '170 color'

# All need left and right hand versions.

aamaoutdir = "/home/timbo/Dropbox/Avian/Technical/Sails/2Rio15/Production_drawings/AAMA"
dmax = 0.2

inputfilename = "/home/timbo/Dropbox/Avian/Technical/Sails/2Rio15/Production_drawings/0Masters/2R15-BS-Master-01.dxf"
d170color = createorclear(aamaoutdir, "170color")
cutlayers, penlayers = ["BS-CUT"], ['BS-PLOT']
outputfilename = os.path.join(d170color, "2R15-BS.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
cutlayers.append('CHEV-CUT')
penlayers.append('CHEV-PLOT')
outputfilename = os.path.join(d170color, "2R15-BS-reflY.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, True, dmax)
;

layerspresent: {'PATCH-PLOT', 'CHEV-PLOT', 'HISTORY', 'BS-PLOT', 'CONSTRUCTION', 'PLOT-OLD', 'BS-CUT', 'PATCH-REF', 'PATCH-CUT', 'CHEV-CUT', 'CUT-OLD'}


AttributeError: module 'ezdxf.math' has no attribute 'vector'

In [7]:
#  2R15-TSF-Master-07.dxf. Top surface front (leading edge). 
#  TSF-OUTER-CUT and TSF-OUTER-PLOT to be cut from 'PXB05' or gitter
#  TSF-INNER-CUT and TSF-INNER-PLOT to be cut from '170color' 
#  PATCH-NORM CUT and PLOT to be from 170color if making a standard glider
#    or PATCH-CARR-CUT and PLOT to be from 170color if making a carrier wing one. 
#  MYLAR-NORM-CUT and PLOT to cut from Mylar if making a standard glider.
#    or MYLAR-CARR-CUT and PLOT to cut from Mylar if making a carrier wing one

aamaoutdir = "/home/timbo/Dropbox/Avian/Technical/Sails/2Rio15/Production_drawings/AAMA"
dmax = 0.2

carrierwingglider = True

inputfilename = "/home/timbo/Dropbox/Avian/Technical/Sails/2Rio15/Production_drawings/0Masters/2R15-TSF-Master-07.dxf"
dPXB05 = createorclear(aamaoutdir, "PXB05")
cutlayers, penlayers = ['TSF-OUTER-CUT'], ['TSF-OUTER-PLOT']
outputfilename = os.path.join(dPXB05, "2R15-TSF-O.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
outputfilename = os.path.join(dPXB05, "2R15-TSF-O-reflY.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, True, dmax)

if carrierwingglider:
    cutlayers, penlayers = ['PATCH-CARR-CUT','TSF-INNER-CUT'], ['PATCH-CARR-PLOT','TSF-INNER-PLOT']
else:
    cutlayers, penlayers = ['PATCH-NORM-CUT','TSF-INNER-CUT'], ['PATCH-NORM-PLOT','TSF-INNER-PLOT']
outputfilename = os.path.join(d170color, "2R15-TSF-PATCH.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
outputfilename = os.path.join(d170color, "2R15-TSF-PATCH-reflY.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, True, dmax)

if carrierwingglider:
    cutlayers, penlayers = ['MYLAR-CARR-CUT'], ['MYLAR-CARR-PLOT']
else:
    cutlayers, penlayers = ['MYLAR-NORM-CUT'], ['MYLAR-NORM-PLOT']
dmylar = createorclear(aamaoutdir, "mylar")
outputfilename = os.path.join(dmylar, "2R15-TSF-MYLAR.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
outputfilename = os.path.join(dmylar, "2R15-TSF-MYLAR-reflY.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, True, dmax)
;

layerspresent: {'PATCH-REF', 'TSF-OUTER-PLOT', 'PATCH-CARR-CUT', 'MYLAR-CARR-PLOT', 'PATCH-CARR-PLOT', 'CONSTRUCTION', 'TSF-OUTER-CUT', 'PATCH-NORM-CUT', 'MYLAR-NORM-CUT', 'HISTORY', 'TSF-INNER-CUT', 'PATCH-NORM-PLOT', 'MYLAR-NORM-PLOT', 'MYLAR-CARR-CUT', 'TSF-INNER-PLOT'}
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppre

''

In [5]:
#  Chevrons-01.dxf
#  CUT to be cut from 'PXB05' or 170color

aamaoutdir = "/home/timbo/Dropbox/Avian/Technical/Sails/2Rio15/Production_drawings/AAMA"
dmax = 0.2

inputfilename = "/home/timbo/Dropbox/Avian/Technical/Sails/2Rio15/Production_drawings/DXF/Chevrons-01.dxf"
cutlayers, penlayers = ['CUT'], []
outputfilename = os.path.join(dPXB05, "Chevrons.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
;

layerspresent: {'CUT'}


''

In [9]:
#  2R15-TSR-TSM-Master-04.dxf. Top surface middle and top surface rear panels. 
#  TSR and TSM cut and plot from either 137cm wide Diax, 144cm wide 170 white or color or 150cm wide ODL06. 
#  Patch cut and patch-plot from 170 color (same as BS).
    
aamaoutdir = "/home/timbo/Dropbox/Avian/Technical/Sails/2Rio15/Production_drawings/AAMA"
dmax = 0.2
b144type = False
 
carrierwingglider = True

inputfilename = "/home/timbo/Dropbox/Avian/Technical/Sails/2Rio15/Production_drawings/0Masters/2R15-TSR-TSM-Master-04.dxf"
ddiax = createorclear(aamaoutdir, "ODL06")



if b144type:
    cutlayers, penlayers = ['TSR-144-CUT'], ['TSR-144-PLOT']
else:
    cutlayers, penlayers = ['TSR-150-CUT'], ['TSR-150-PLOT']
outputfilename = os.path.join(ddiax, "2R15-TSR.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
outputfilename = os.path.join(ddiax, "2R15-TSR-reflY.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, True, dmax)

if b144type:
    cutlayers, penlayers = ['TSM-144-CUT'], ['TSM-144-PLOT']
else:
    cutlayers, penlayers = ['TSM-150-CUT'], ['TSM-150-PLOT']
outputfilename = os.path.join(ddiax, "2R15-TSM.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
outputfilename = os.path.join(ddiax, "2R15-TSM-reflY.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, True, dmax)

print("writing to", d170color)
cutlayers, penlayers = ['PATCH-CUT'], ['PATCH-PLOT']
outputfilename = os.path.join(d170color, "2R15-PATCH.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
outputfilename = os.path.join(d170color, "2R15-PATCH-reflY.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, True, dmax)
;

layerspresent: {'PATCH-REF', 'TSM-150-PLOT', 'TSR-144-PLOT', 'PATCH-CUT', 'TSM-144-CUT', 'PATCH-PLOT', 'TSR-150-CUT', 'TSR-150-PLOT', 'TSM-144-PLOT', 'TSM-150-CUT', 'TSR-144-CUT', '0'}
dmaxs-tail:  [1.3194022429241056e-10, 0.002025941914434135, 0.15062684391094103]
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
suppressing loop making merge
layerspresent: {'PATCH-REF', 'TSM-150-PLOT', 'TSR-144-PLOT', 'PATCH-CUT', 'TSM-144-CUT', 'PATCH-PLOT', 'TSR-150-CUT', 'TSR-150-PLOT', 'TSM-144-PLOT', 'TSM-150-CUT', 'TSR-144-CUT', '0'}
dmaxs-tail:  [1.3194022429241056e-10, 0.002025941914434135, 0.15062684391094103]
suppressing loop making merge
suppressing loop m

''

In [20]:
#-2R15_Carrier_Main_bag-01-. 
#  GREY_CUT and GREY_PLOT to be cut and plotted from grey fabric, BLUE_CUT and BLUE_PLOT for blue fabric

aamaoutdir = "/home/timbo/Dropbox/Avian/Technical/Sails/2Rio15/Production_drawings/AAMA"
dmax = 0.2

inputfilename = "/home/timbo/Dropbox/Avian/Technical/Sails/Bags/2R15_Carrier_Main_bag-01.dxf"
dgreybag = createorclear(aamaoutdir, "greybag")
cutlayers, penlayers = ["GREY_CUT"], ['GREY_PLOT']
outputfilename = os.path.join(dgreybag, "main-grey.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)

dbluebag = createorclear(aamaoutdir, "bluebag")
cutlayers, penlayers = ["BLUE_CUT"], ['BLUE_PLOT']
outputfilename = os.path.join(dbluebag, "main-blue.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
;

layerspresent: {'BLUE_CUT', 'BLUE_PLOT', 'GREY_CUT', 'GREY_PLOT', '0'}
layerspresent: {'BLUE_CUT', 'BLUE_PLOT', 'GREY_CUT', 'GREY_PLOT', '0'}


''

In [21]:
#-2R15_Carrier_bits_bag-02.
#  GREY_CUT and GREY_PLOT to be cut and plotted from grey fabric, BLUE_CUT and BLUE_PLOT for blue fabric

aamaoutdir = "/home/timbo/Dropbox/Avian/Technical/Sails/2Rio15/Production_drawings/AAMA"
dmax = 0.2

inputfilename = "/home/timbo/Dropbox/Avian/Technical/Sails/Bags/2R15_Carrier_bits_bag-02.dxf"

cutlayers, penlayers = ["GREY_CUT"], ['GREY_PLOT']
outputfilename = os.path.join(dgreybag, "bits-grey.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)

cutlayers, penlayers = ["BLUE_CUT"], ['BLUE_PLOT']
outputfilename = os.path.join(dbluebag, "bits-blue.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)
;

layerspresent: {'GREY_CUT', 'BLUE_PLOT', 'GREY_PLOT', 'BLUE_CUT'}
layerspresent: {'GREY_CUT', 'BLUE_PLOT', 'GREY_PLOT', 'BLUE_CUT'}


''

In [9]:
#Fly15 Keel pocket, no reflection needed

inputfilename = "../dxffiles/F15-KP-01.dxf"
cutlayers, penlayers = ["KP-CUT"], ['KP-PLOT']
outputfilename = os.path.join(d210mastsleeve, "F15-KP.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)

layerspresent: {'KP-CUT', 'KP-PLOT'}


[([<class 'ezdxf.entities.arc.Arc'> ARC(#55),
   <class 'ezdxf.entities.line.Line'> LINE(#54),
   <class 'ezdxf.entities.arc.Arc'> ARC(#53),
   <class 'ezdxf.entities.line.Line'> LINE(#52),
   <class 'ezdxf.entities.arc.Arc'> ARC(#51),
   <class 'ezdxf.entities.line.Line'> LINE(#50),
   <class 'ezdxf.entities.arc.Arc'> ARC(#4F),
   <class 'ezdxf.entities.line.Line'> LINE(#67),
   <class 'ezdxf.entities.line.Line'> LINE(#66),
   <class 'ezdxf.entities.line.Line'> LINE(#68),
   <class 'ezdxf.entities.line.Line'> LINE(#62),
   <class 'ezdxf.entities.arc.Arc'> ARC(#59),
   <class 'ezdxf.entities.line.Line'> LINE(#5A),
   <class 'ezdxf.entities.arc.Arc'> ARC(#5B),
   <class 'ezdxf.entities.line.Line'> LINE(#5C),
   <class 'ezdxf.entities.arc.Arc'> ARC(#5D),
   <class 'ezdxf.entities.line.Line'> LINE(#5E),
   <class 'ezdxf.entities.arc.Arc'> ARC(#5F),
   <class 'ezdxf.entities.line.Line'> LINE(#60),
   <class 'ezdxf.entities.arc.Arc'> ARC(#61),
   <class 'ezdxf.entities.line.Line'> LINE(#63)

In [11]:
#Pen alignment test, no reflection needed

inputfilename = "../dxffiles/Pen-align-test.dxf"
cutlayers, penlayers = ["CUT"], ['PLOT']
outputfilename = os.path.join(d170color, "test.dxf")
convertall(inputfilename, outputfilename, cutlayers, penlayers, False, dmax)

layerspresent: {'CUT', 'PLOT'}


[([<class 'ezdxf.entities.line.Line'> LINE(#20015),
   <class 'ezdxf.entities.line.Line'> LINE(#20014),
   <class 'ezdxf.entities.line.Line'> LINE(#20013),
   <class 'ezdxf.entities.line.Line'> LINE(#20016)],
  [False, False, False, False],
  [],
  [<class 'ezdxf.entities.circle.Circle'> CIRCLE(#50),
   <class 'ezdxf.entities.circle.Circle'> CIRCLE(#51),
   <class 'ezdxf.entities.circle.Circle'> CIRCLE(#52),
   <class 'ezdxf.entities.circle.Circle'> CIRCLE(#53),
   <class 'ezdxf.entities.circle.Circle'> CIRCLE(#54),
   <class 'ezdxf.entities.line.Line'> LINE(#55),
   <class 'ezdxf.entities.line.Line'> LINE(#56)])]